In [3]:
#importing
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

d:\project\Sanjivani-Medical_Chat_Bot\mchat\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
PINECONE_API_KEY = "aebed40e-c461-4790-a920-02825138142e"
PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF

def load_pdf(data):
    loader = DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyMuPDFLoader)
    
    documents = loader.load()
    
    return documents

In [6]:
extracted_data = load_pdf("../data/")

In [8]:
#extracted_data

In [9]:
#Create Text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,
                                   chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5779


In [12]:
#download embedding Model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

.gitattributes: 100%|██████████| 1.18k/1.18k [00:00<00:00, 792kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<?, ?B/s] 
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 1.71MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 40.8kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 223kB/s]
pytorch_model.bin: 100%|██████████| 90.9M/90.9M [00:15<00:00, 5.94MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 20.6kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 30.4kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 800kB/s]
tokenizer_config.json: 100%|██████████| 350/350 [00:00<00:00, 87.5kB/s]
train_script.py: 100%|██████████| 13.2k/13.2k [00:00<00:00, 4.45MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 622kB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 113kB/s]


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [15]:
#test embbedings
query_result = embeddings.embed_query("prashant is Boss")
print("length", len(query_result))

length 384


In [16]:
query_result

[-0.009107615798711777,
 0.05873565375804901,
 -0.07030512392520905,
 -0.056879185140132904,
 -0.0031525169033557177,
 0.05615537613630295,
 0.10656753182411194,
 0.050294119864702225,
 -0.014334275387227535,
 0.005456308368593454,
 0.009563865140080452,
 -0.02699340134859085,
 -0.05469213053584099,
 -0.037342533469200134,
 0.006810941733419895,
 0.0102402213960886,
 0.017730407416820526,
 0.07305754721164703,
 0.057079024612903595,
 -0.015326074324548244,
 -0.02144295908510685,
 -0.055910635739564896,
 -0.03386329859495163,
 -0.04223000630736351,
 -0.0162084698677063,
 0.03557737544178963,
 -0.03885449469089508,
 0.0957055315375328,
 0.03537826985120773,
 -0.07849211990833282,
 0.037105198949575424,
 -0.001891199266538024,
 0.0255318321287632,
 -0.05217335373163223,
 -0.048388756811618805,
 0.0839272066950798,
 -0.06922049075365067,
 0.0422193668782711,
 -0.07365908473730087,
 0.004206219222396612,
 0.04933784157037735,
 -0.0028113722801208496,
 -0.07131317257881165,
 -0.0269606206566

In [17]:
#initializing the Pinecone

pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name = "sanjivani-med-chatbot"

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],
                                embeddings,
                                index_name=index_name)

In [18]:
#if we have index we can load it like this

docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What is Allergies"

docs = docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='to commonly encountered environmental substances.\nPurpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of', metadata={}), Document(page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depend on the tissues most sensi-\ntive to the aller

In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:
PROMPT = PromptTemplate(template=prompt_template,
                        input_variables=["context","question"])
chain_type_kwargs = {"prompt": PROMPT}

In [23]:
llm = CTransformers(model = "model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config = {"max_new_tokens": 512,
                              "temperature":0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-658bdb15-0ec83c1a4309018d27c68f21;15789320-46a3-4d33-bc24-96a875fb4a64)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.